In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
distance_first_camera = 3000.0

In [3]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Create an engine instance
alchemy_engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Connect to PostgreSQL server
db_connection = alchemy_engine.connect()

# Read the data into a_df
cameras_df = pd.read_sql(f"select * from \"project\".\"Cameras\"", db_connection)


In [4]:
cameras_df.head()

,id_meetlocatie,start_locatie_longitude,start_locatie_latitude,wegnaam_vild,rijstrook_rijbaan,segment
0,GEO0B_R_RWSTI357698,4.537880,51.927530,Ring Rotterdam,lane1,south
1,GEO0K_K_RWSTI357132,4.366503,51.922442,Ring Rotterdam,lane1,north
2,GEO0K_K_RWSTI357777,4.565618,51.874718,Ring Rotterdam,lane1,south
3,RWS01_MONIBAS_0040vwe0763ra,4.384620,51.879240,Ring Rotterdam,lane1,north
4,RWS01_MONIBAS_0040vwn0713ra,4.368560,51.914020,Ring Rotterdam,lane1,north


get 8 lane segments with cameras

In [5]:
import numpy as np

In [6]:
def haversine(lon1, lat1, lon2, lat2):
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371 # Radius of Earth in kilometers. Use 3956 for miles
    return c * r * 1000

In [7]:
# clock-wise lane was taken. 
first_camera_proportion = 5/12
second_camera_proportion = 7/12
north_cameras_df = cameras_df[cameras_df['segment'] == 'west-east']
south_cameras_df = cameras_df[cameras_df['segment'] == 'east-west']
east_cameras_df = cameras_df[cameras_df['segment'] == 'south']
west_cameras_df = cameras_df[cameras_df['segment'] == 'north']

print('north_cameras_df shape: ', north_cameras_df.shape)

# now the cameras need to be sorted. For north, the cameras_df start_locatie_latitude should be increasing
north_cameras_df = north_cameras_df.sort_values(by=['start_locatie_latitude'])
# for south, the start_locatie_latitude should be decreasing
south_cameras_df = south_cameras_df.sort_values(by=['start_locatie_latitude'], ascending=False)
# for east, the start_locatie_longitude should be decreasing
east_cameras_df = east_cameras_df.sort_values(by=['start_locatie_longitude'], ascending=False)
# for west, the start_locatie_longitude should be increasing
west_cameras_df = west_cameras_df.sort_values(by=['start_locatie_longitude'])

# --------- the cameras are now sorted for their order, which means the lowest index is earlier 
# --------- in the road segment. Now we need to find the starting point for each side
# --------- we will add cameras with their distance to the segment until the distance from 
# --------- the starting point is 3000 meters.

# for every side, we take 2 cameras as the starting points, which is the camera on 5/12 and 7/12 of total cameras for the side
starting_point_camera_id_s_north = [int(len(north_cameras_df)*first_camera_proportion), int(len(north_cameras_df)*second_camera_proportion)]

starting_point_camera_id_s_south = [int(len(south_cameras_df)*first_camera_proportion), int(len(south_cameras_df)*second_camera_proportion)]

starting_point_camera_id_s_east = [int(len(east_cameras_df)*first_camera_proportion), int(len(east_cameras_df)*second_camera_proportion)]

starting_point_camera_id_s_west = [int(len(west_cameras_df)*first_camera_proportion), int(len(west_cameras_df)*second_camera_proportion)]

print(starting_point_camera_id_s_north)

def process_camera_data(camera_index, cameras_data_frame):
    # Initialize DataFrame for road segments
    road_segments_df = pd.DataFrame(columns=['segment_id', 'cameras_before', 'cameras_after'])

    # Reference to the specific camera
    camera = cameras_data_frame.iloc[camera_index]

    

    # Process cameras before the specified index
    index = camera_index
    cameras_before_json = {}
    # add the first camera
    cameras_before_json[camera['id_meetlocatie']] = distance_first_camera
    accumulated_distance = 0.0
    while accumulated_distance < distance_first_camera:
        previous_camera = cameras_data_frame.iloc[index]
        index -= 1
        if index < 0:
            break
        camera = cameras_data_frame.iloc[index]
        accumulated_distance += haversine(previous_camera['start_locatie_longitude'], previous_camera['start_locatie_latitude'], camera['start_locatie_longitude'], camera['start_locatie_latitude'])
        if (accumulated_distance <= distance_first_camera):
            cameras_before_json[camera['id_meetlocatie']] = (distance_first_camera - accumulated_distance)

    # Process cameras after the specified index
    index = camera_index
    cameras_after_json = {}
    accumulated_distance = 0.0
    while accumulated_distance < distance_first_camera:
        previous_camera = cameras_data_frame.iloc[index]
        index += 1
        if index >= len(cameras_data_frame):
            break
        camera = cameras_data_frame.iloc[index]
        accumulated_distance += haversine(previous_camera['start_locatie_longitude'], previous_camera['start_locatie_latitude'], camera['start_locatie_longitude'], camera['start_locatie_latitude'])
        if accumulated_distance <= distance_first_camera:
            cameras_after_json[camera['id_meetlocatie']] = (distance_first_camera + accumulated_distance)

    segment_id = "no_incident_camera_base" + str(camera['id_meetlocatie'])
    # Creating a new DataFrame row and concatenating it to road_segments_df
    new_row = pd.DataFrame([{
        'segment_id': segment_id,
        'cameras_before': cameras_before_json,
        'cameras_after': cameras_after_json
    }])
    road_segments_df = pd.concat([road_segments_df, new_row], ignore_index=True)
    return road_segments_df

# create north road_segment df
north_camera_index = starting_point_camera_id_s_north[0]
road_segments_df = process_camera_data(north_camera_index, north_cameras_df)

north_camera_index = starting_point_camera_id_s_north[1]
new_road_segments_df = process_camera_data(north_camera_index, north_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

# create south road_segment df
south_camera_index = starting_point_camera_id_s_south[0]
new_road_segments_df = process_camera_data(south_camera_index, south_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

south_camera_index = starting_point_camera_id_s_south[1]
new_road_segments_df = process_camera_data(south_camera_index, south_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

# create east road_segment df
east_camera_index = starting_point_camera_id_s_east[0]
new_road_segments_df = process_camera_data(east_camera_index, east_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

east_camera_index = starting_point_camera_id_s_east[1]
new_road_segments_df = process_camera_data(east_camera_index, east_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

# create west road_segment df
west_camera_index = starting_point_camera_id_s_west[0]
new_road_segments_df = process_camera_data(west_camera_index, west_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

west_camera_index = starting_point_camera_id_s_west[1]
new_road_segments_df = process_camera_data(west_camera_index, west_cameras_df)
road_segments_df = pd.concat([road_segments_df, new_road_segments_df], ignore_index=True)

print(road_segments_df.head(100))



    
    






north_cameras_df shape:  (43, 6)
[17, 25]
                                          segment_id  \
0  no_incident_camera_baseRWS01_MONIBAS_0201hrr03...   
1  no_incident_camera_baseRWS01_MONIBAS_0201hrr03...   
2  no_incident_camera_baseRWS01_MONIBAS_0150vwn05...   
3  no_incident_camera_baseRWS01_MONIBAS_0150vwy05...   
4  no_incident_camera_baseRWS01_MONICA_10D01002F4...   
5  no_incident_camera_baseRWS01_MONIBAS_0160vwx01...   
6  no_incident_camera_baseRWS01_MONIBAS_0150vws05...   
7  no_incident_camera_baseRWS01_MONIBAS_0150vws05...   

                                      cameras_before  \
0  {'RWS01_MONIBAS_0201hrr0285ra': 3000.0, 'RWS01...   
1  {'RWS01_MONIBAS_0201hrr0309ra': 3000.0, 'RWS01...   
2            {'RWS01_MONIBAS_0150vwy0544ra': 3000.0}   
3  {'RWS01_MONIBAS_0150vwn0602ra': 3000.0, 'RWS01...   
4  {'RWS01_MONIBAS_0160vwx0202ra': 3000.0, 'RWS01...   
5  {'RWS01_MONICA_10D01002C80DB8200007': 3000.0, ...   
6  {'RWS01_MONICA_10D0040B680668200005': 3000.0, ...   
7  {'

In [8]:
# let pandas not truncate the output
pd.set_option('display.max_colwidth', None)


In [9]:


road_segments_df.head(100)

,segment_id,cameras_before,cameras_after
0,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra,"{'RWS01_MONIBAS_0201hrr0285ra': 3000.0, 'RWS01_MONIBAS_0201hrr0284ra': 2839.976189568402, 'RWS01_MONIBAS_0201hrr0257ra': 431.05659007992244}","{'RWS01_MONIBAS_0201hrr0288ra': 3231.814935862123, 'RWS01_MONIBAS_0201hrr0291ra': 3533.7924432076416, 'RWS01_MONIBAS_0201hrr0295ra': 3924.3419620541467, 'RWS01_MONIBAS_0201hrr0298ra': 4231.080458587956, 'RWS01_MONIBAS_0201hrr0301ra': 4614.416258398908, 'RWS01_MONIBAS_0200vwa0303ra': 4743.437957210798, 'RWS01_MONIBAS_0201hrr0306ra': 5026.462277661043, 'RWS01_MONIBAS_0201hrr0309ra': 5396.736692551054, 'RWS01_MONIBAS_0201hrr0312ra': 5673.210200071818, 'RWS01_MONIBAS_0201hrr0315ra': 5961.9439339821965, 'RWS01_MONIBAS_0201hrr0319ra': 6311.120786673314}"
1,no_incident_camera_baseRWS01_MONIBAS_0201hrr0343ra,"{'RWS01_MONIBAS_0201hrr0309ra': 3000.0, 'RWS01_MONIBAS_0201hrr0306ra': 2629.725585109989, 'RWS01_MONIBAS_0200vwa0303ra': 2346.701264659744, 'RWS01_MONIBAS_0201hrr0301ra': 2217.679565847854, 'RWS01_MONIBAS_0201hrr0298ra': 1834.343766036901, 'RWS01_MONIBAS_0201hrr0295ra': 1527.6052695030921, 'RWS01_MONIBAS_0201hrr0291ra': 1137.0557506565872, 'RWS01_MONIBAS_0201hrr0288ra': 835.0782433110685, 'RWS01_MONIBAS_0201hrr0285ra': 603.2633074489454, 'RWS01_MONIBAS_0201hrr0284ra': 443.2394970173473}","{'RWS01_MONIBAS_0201hrr0312ra': 3276.473507520763, 'RWS01_MONIBAS_0201hrr0315ra': 3565.2072414311415, 'RWS01_MONIBAS_0201hrr0319ra': 3914.3840941222593, 'RWS01_MONIBAS_0201hrr0322ra': 4223.191397121996, 'RWS01_MONIBAS_0201hrr0324ra': 4476.743906641348, 'RWS01_MONIBAS_0201hrr0328ra': 4808.485889682265, 'RWS01_MONIBAS_0201hrr0332ra': 5207.043163254888, 'RWS01_MONIBAS_0201hrr0333ra': 5365.711427065204, 'RWS01_MONIBAS_0201hrr0338ra': 5855.304155204734, 'RWS01_MONIBAS_0201hrr0343ra': 6304.468399547873}"
2,no_incident_camera_baseRWS01_MONIBAS_0150vwn0594ra,{'RWS01_MONIBAS_0150vwy0544ra': 3000.0},"{'RWS01_MONIBAS_0150vwy0548ra': 3411.2182018624626, 'RWS01_MONIBAS_0150vwn0594ra': 7829.7709815225735}"
3,no_incident_camera_baseRWS01_MONIBAS_0150vwy0553ra,"{'RWS01_MONIBAS_0150vwn0602ra': 3000.0, 'RWS01_MONIBAS_0151hrl0619ra': 1428.2844442510773}","{'RWS01_MONIBAS_0150vwn0599ra': 3352.261196874741, 'RWS01_MONIBAS_0150vwn0584ra': 4802.313378833861, 'RWS01_MONIBAS_0150vwy0553ra': 7872.301196693836}"
4,no_incident_camera_baseRWS01_MONICA_10D01002F400B8200007,"{'RWS01_MONIBAS_0160vwx0202ra': 3000.0, 'RWS01_MONICA_10D010033400B8200007': 2698.4918461856078, 'RWS01_MONIBAS_0160vwx0211ra': 2096.8796960450863, 'RWS01_MONIBAS_0160vwx0215ra': 1696.3975021464257, 'RWS01_MONIBAS_0160vwx0219ra': 1298.6502411816257}","{'RWS01_MONIBAS_0160vwx0182ra': 4888.480444419746, 'RWS01_MONICA_10D01002C80DB8200007': 5291.0434537363135, 'GEO0B_R_RWSTI357698': 5577.955861542538, 'RWS01_MONIBAS_0160vwx0185ra': 5999.572952666786, 'RWS01_MONICA_10D01002F400B8200007': 6358.708547755443}"
5,no_incident_camera_baseRWS01_MONIBAS_0160vwx0193ra,"{'RWS01_MONICA_10D01002C80DB8200007': 3000.0, 'RWS01_MONIBAS_0160vwx0182ra': 2597.4369906834327, 'RWS01_MONIBAS_0160vwx0202ra': 708.9565462636865, 'RWS01_MONICA_10D010033400B8200007': 407.44839244929426}","{'GEO0B_R_RWSTI357698': 3286.9124078062237, 'RWS01_MONIBAS_0160vwx0185ra': 3708.529498930472, 'RWS01_MONICA_10D01002F400B8200007': 4067.6650940191294, 'RWS01_MONIBAS_0160vwx0193ra': 4479.8323923520575}"
6,no_incident_camera_baseRWS01_MONIBAS_0150vws0507ra,"{'RWS01_MONICA_10D0040B680668200005': 3000.0, 'RWS01_MONIBAS_0040vwn0728ra': 2876.5475929801537, 'RWS01_MONICA_10D0040B403D68200005': 2069.223161846097, 'RWS01_MONIBAS_0040vwn0723ra': 1755.6347090412992, 'RWS01_MONICA_10D0040B5C2468200005': 1402.9759157686697, 'RWS01_MONICA_10D0040B545B68200005': 1259.4294929340037, 'RWS01_MONIBAS_0040vwn0717ra': 450.96131796663485, 'RWS01_MONIBAS_0040vwn0713ra': 55.52589440530437}","{'RWS01_MONICA_10D0040B6C3868200005': 3148.66096323054, 'RWS01_MONICA_10D0040B705468200005': 3275.8469075024036, 'RWS01_MONIBAS_0040vwn0734ra': 3404.63

In [10]:
# retrieve the incident data from the database, to create a list of dates which need to be excluded for data generation
import pandas as pd
from sqlalchemy import create_engine
import json

# Database connection parameters
driver = 'postgresql'
username = 'dab_ds23241a_223'
password = 'Bh6KFgcWazMMCPvZ'
server = 'bronto.ewi.utwente.nl'
port = '5432'
dbname = username  # dbname is the same as username

# Create the database engine
engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')

# Load the 'incidents' table into a DataFrame
incidents_df = pd.read_sql_table('Incidents_With_Cameras', engine, schema='project')

incidents_df.head()


,id,starttijd,type,gedetailleerd_type,primaire_locatie_lengtegraad,primaire_locatie_breedtegraad,segment,before,after
0,RWS02_0000157188_157188,2020-03-01 08:38:28,vehicle_obstruction,emergencyVehicle,4.366420,51.920952,north,"{""RWS01_MONIBAS_0041hrl0706ra"": 0.022954588858243016, ""RWS01_MONIBAS_0041hrl0710ra"": 0.45393771470169625, ""RWS01_MONIBAS_0040vwn0713ra"": 0.7846379079863572, ""RWS01_MONIBAS_0040vwn0717ra"": 1.1785015400322572, ""RWS01_MONICA_10D0040B403D68200005"": 1.4702194168439926, ""RWS01_MONIBAS_0040vwn0723ra"": 1.776698914563434, ""RWS01_MONICA_10D0040B545B68200005"": 1.9800096045727866}","{""GEO0K_K_RWSTI357132"": 0.16579609461072695}"
1,RWS02_0000157197_157197,2020-03-01 10:00:57,vehicle_obstruction,emergencyVehicle,4.432840,51.928864,west-east,"{""RWS01_MONIBAS_0201hrr0284ra"": 0.0942395501543207, ""RWS01_MONIBAS_0201hrr0281ra"": 0.4066900837525745, ""RWS01_MONICA_10D01404545AD0070007"": 0.7179153032241979, ""RWS01_MONIBAS_0201hrr0274ra"": 1.0204058689070725, ""RWS01_MONIBAS_0201hrr0272ra"": 1.270913474562368, ""RWS01_MONIBAS_0201hrr0269ra"": 1.5503867551716262, ""RWS01_MONIBAS_0201hrr0266ra"": 1.7377148766974488, ""RWS01_MONIBAS_0201hrr0264ra"": 1.8814637977780844}","{""RWS01_MONIBAS_0201hrr0285ra"": 0.06578558673681623, ""RWS01_MONIBAS_0201hrr0288ra"": 0.29760052133343573, ""RWS01_MONIBAS_0201hrr0291ra"": 0.5991838927093442, ""RWS01_MONIBAS_0201hrr0295ra"": 0.989233553421513, ""RWS01_MONIBAS_0201hrr0298ra"": 1.2927062743048217, ""RWS01_MONIBAS_0201hrr0301ra"": 1.6692123031028396, ""RWS01_MONIBAS_0200vwa0303ra"": 1.7981440786338454, ""RWS01_MONIBAS_0201hrr0306ra"": 2.080692616215798, ""RWS01_MONIBAS_0201hrr0309ra"": 2.4509285065594817, ""RWS01_MONIBAS_0201hrr0312ra"": 2.7217936968367473, ""RWS01_MONIBAS_0201hrr0315ra"": 3.0004190334589738, ""RWS01_MONIBAS_0201hrr0319ra"": 3.339240453292579, ""RWS01_MONIBAS_0201hrr0322ra"": 3.639016640617847, ""RWS01_MONIBAS_0201hrr0324ra"": 3.8847881162630693, ""RWS01_MONIBAS_0201hrr0328ra"": 4.214035964741166, ""RWS01_MONIBAS_0201hrr0332ra"": 4.606036000125693, ""RWS01_MONIBAS_0201hrr0333ra"": 4.76327632471753}"
2,RWS02_0000157269_157269,2020-03-01 13:21:58,vehicle_obstruction,emergencyVehicle,4.456025,51.864422,east-west,"{""RWS01_MONIBAS_0150vwy0557ra"": 0.24950501028517727, ""RWS01_MONIBAS_0150vwy0560ra"": 0.5399819637446478, ""RWS01_MONIBAS_0150vwq0565ra"": 1.017660734587508, ""RWS01_MONIBAS_0150vwq0567ra"": 1.231012659270556, ""RWS01_MONIBAS_0150vwq0572ra"": 1.7072070176277643}","{""RWS01_MONIBAS_0150vwy0553ra"": 0.2295080957989754, ""RWS01_MONIBAS_0150vwy0548ra"": 0.6402749946222849, ""RWS01_MONIBAS_0150vwy0544ra"": 1.0514929992528421, ""RWS01_MONIBAS_0150vwy0540ra"": 1.428615558439161, ""RWS01_MONIBAS_0150vwy0538ra"": 1.6969940787257678, ""RWS01_MONIBAS_0150vwy0533ra"": 2.143080873776183, ""RWS01_MONIBAS_0150vwy0531ra"": 2.3710559049436304, ""RWS01_MONIBAS_0150vwy0526ra"": 2.8264154304542046, ""RWS01_MONIBAS_0150vwy0523ra"": 3.227903778094212, ""RWS01_MONIBAS_0150vwy0520ra"": 3.4963748091212725, ""RWS01_MONIBAS_0150vwy0517ra"": 3.7457085243780948, ""RWS01_MONIBAS_0150vwq0512ra"": 4.2654490521006325}"
3,RWS03_816372_1,2020-03-01 15:19:57,vehicle_obstruction,brokenDownVehicle,4.538568,51.909523,south,"{""RWS01_MONIBAS_0160vwx0193ra"": 0.896763594374343, ""RWS01_MONICA_10D01002F400B8200007"": 1.2820849504377159, ""RWS01_MONIBAS_0160vwx0185ra"": 1.5938539374601517, ""RWS01_MONIBAS_0160vwx0182ra"": 1.8870175253928867}","{""RWS01_MONIBAS_0160vwx0202ra"": 0.0016637962861347836, ""RWS01_MONICA_10D010033400B8200007"": 0.3031620285169147, ""RWS01_MONIBAS_0160vwx0211ra"": 0.9047678170560078, ""RWS01_MONIBAS_0160vwx0215ra"": 1.3040409410501481, ""RWS01_MONIBAS_0160vwx0219ra"": 1.6722559780841995, ""GEO0K_K_RWSTI357777"": 4.292281864912008}"
4,RWS02_0000157393_157393,2020-03-01 23:03:29,vehicle_obstruction,emergencyVehicle,4.439620,51.868599,east-west,"{""RWS01_MONIBAS_0150vwy0544ra"": 0.16698020032248845, ""RWS01_MONIBAS_0150vwy0548ra"": 0.5781552253280561, ""RWS01_MONIBAS_01

In [11]:
dates_incident_happened = pd.to_datetime(incidents_df['starttijd'].dt.strftime('%Y-%m-%d %H:%M')).unique()


In [12]:
#choose random time between 00:00 and 20:00 to start capturing data for the road segments
import random
import datetime

random_start_date_times = []

while len(random_start_date_times) < 80:
    # Random start time within the day range
    start_date = datetime.datetime(2020, 3, random.randint(1,28), random.randint(0, 20), random.randint(0, 59), random.randint(0, 59))
    
    # Check if the date is not closer than 3 hours to any incident date time
    was_not_close_to_incident = True
    for incident_datetime in dates_incident_happened:
        print('incident datetime: ', incident_datetime)
        # Calculate the absolute difference in seconds and convert to hours
        time_difference = abs((start_date - incident_datetime).total_seconds()) / 3600
        if time_difference < 3:
            was_not_close_to_incident = False
            break
    if was_not_close_to_incident:
        print(start_date)  # Prints the full datetime, not just the date
        random_start_date_times.append(start_date)

print("Total dates generated:", len(random_start_date_times))
print("Sample of generated dates:", random_start_date_times[:5])


incident datetime:  2020-03-01 08:38:00
incident datetime:  2020-03-01 10:00:00
incident datetime:  2020-03-01 13:21:00
incident datetime:  2020-03-01 15:19:00
incident datetime:  2020-03-01 23:03:00
incident datetime:  2020-03-02 09:17:00
incident datetime:  2020-03-02 13:00:00
incident datetime:  2020-03-02 22:36:00
incident datetime:  2020-03-03 02:23:00
incident datetime:  2020-03-03 09:17:00
incident datetime:  2020-03-03 11:25:00
incident datetime:  2020-03-04 07:44:00
incident datetime:  2020-03-04 14:33:00
incident datetime:  2020-03-04 16:01:00
incident datetime:  2020-03-04 16:57:00
incident datetime:  2020-03-04 19:25:00
incident datetime:  2020-03-06 16:29:00
incident datetime:  2020-03-05 11:25:00
incident datetime:  2020-03-05 19:48:00
incident datetime:  2020-03-06 08:00:00
incident datetime:  2020-03-06 08:37:00
incident datetime:  2020-03-06 13:51:00
incident datetime:  2020-03-06 14:17:00
incident datetime:  2020-03-06 14:34:00
incident datetime:  2020-04-10 15:32:00


In [13]:
# for each road segment, add 10 entries with different 'starttijd' , which is in the format: 2020-03-01 23:03:29 , and comes from the random_start_date_times list
# set no_incident_id to the segment_id + the starttijd

# Initialize DataFrame for road segments
road_segments_data_df = pd.DataFrame(columns=['no_incident_id', 'starttijd', 'before', 'after'])
start_date_index = 0

for index, row in road_segments_df.iterrows():
    for i in range(10):
        start_date_time = random_start_date_times[start_date_index]
        start_date_index += 1
        segment_id = row['segment_id']
        no_incident_id = segment_id + '_' + str(start_date_time)
        new_row = pd.DataFrame([{
            'no_incident_id': no_incident_id,
            'starttijd': start_date_time,
            'before': json.dumps(row['cameras_before']),
            'after': json.dumps(row['cameras_after'])
        }])
        road_segments_data_df = pd.concat([road_segments_data_df, new_row], ignore_index=True)
        
road_segments_data_df.shape



# rename no_incident_id to id
road_segments_data_df = road_segments_data_df.rename(columns={'no_incident_id': 'id'})

road_segments_data_df


/var/folders/c0/2gmfm5qd4tl9nzl882r0xr040000gn/T/ipykernel_61666/2110827403.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  road_segments_data_df = pd.concat([road_segments_data_df, new_row], ignore_index=True)


,id,starttijd,before,after
0,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-10 03:18:55,2020-03-10 03:18:55,"{""RWS01_MONIBAS_0201hrr0285ra"": 3000.0, ""RWS01_MONIBAS_0201hrr0284ra"": 2839.976189568402, ""RWS01_MONIBAS_0201hrr0257ra"": 431.05659007992244}","{""RWS01_MONIBAS_0201hrr0288ra"": 3231.814935862123, ""RWS01_MONIBAS_0201hrr0291ra"": 3533.7924432076416, ""RWS01_MONIBAS_0201hrr0295ra"": 3924.3419620541467, ""RWS01_MONIBAS_0201hrr0298ra"": 4231.080458587956, ""RWS01_MONIBAS_0201hrr0301ra"": 4614.416258398908, ""RWS01_MONIBAS_0200vwa0303ra"": 4743.437957210798, ""RWS01_MONIBAS_0201hrr0306ra"": 5026.462277661043, ""RWS01_MONIBAS_0201hrr0309ra"": 5396.736692551054, ""RWS01_MONIBAS_0201hrr0312ra"": 5673.210200071818, ""RWS01_MONIBAS_0201hrr0315ra"": 5961.9439339821965, ""RWS01_MONIBAS_0201hrr0319ra"": 6311.120786673314}"
1,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-25 07:26:02,2020-03-25 07:26:02,"{""RWS01_MONIBAS_0201hrr0285ra"": 3000.0, ""RWS01_MONIBAS_0201hrr0284ra"": 2839.976189568402, ""RWS01_MONIBAS_0201hrr0257ra"": 431.05659007992244}","{""RWS01_MONIBAS_0201hrr0288ra"": 3231.814935862123, ""RWS01_MONIBAS_0201hrr0291ra"": 3533.7924432076416, ""RWS01_MONIBAS_0201hrr0295ra"": 3924.3419620541467, ""RWS01_MONIBAS_0201hrr0298ra"": 4231.080458587956, ""RWS01_MONIBAS_0201hrr0301ra"": 4614.416258398908, ""RWS01_MONIBAS_0200vwa0303ra"": 4743.437957210798, ""RWS01_MONIBAS_0201hrr0306ra"": 5026.462277661043, ""RWS01_MONIBAS_0201hrr0309ra"": 5396.736692551054, ""RWS01_MONIBAS_0201hrr0312ra"": 5673.210200071818, ""RWS01_MONIBAS_0201hrr0315ra"": 5961.9439339821965, ""RWS01_MONIBAS_0201hrr0319ra"": 6311.120786673314}"
2,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-22 06:43:22,2020-03-22 06:43:22,"{""RWS01_MONIBAS_0201hrr0285ra"": 3000.0, ""RWS01_MONIBAS_0201hrr0284ra"": 2839.976189568402, ""RWS01_MONIBAS_0201hrr0257ra"": 431.05659007992244}","{""RWS01_MONIBAS_0201hrr0288ra"": 3231.814935862123, ""RWS01_MONIBAS_0201hrr0291ra"": 3533.7924432076416, ""RWS01_MONIBAS_0201hrr0295ra"": 3924.3419620541467, ""RWS01_MONIBAS_0201hrr0298ra"": 4231.080458587956, ""RWS01_MONIBAS_0201hrr0301ra"": 4614.416258398908, ""RWS01_MONIBAS_0200vwa0303ra"": 4743.437957210798, ""RWS01_MONIBAS_0201hrr0306ra"": 5026.462277661043, ""RWS01_MONIBAS_0201hrr0309ra"": 5396.736692551054, ""RWS01_MONIBAS_0201hrr0312ra"": 5673.210200071818, ""RWS01_MONIBAS_0201hrr0315ra"": 5961.9439339821965, ""RWS01_MONIBAS_0201hrr0319ra"": 6311.120786673314}"
3,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-07 01:21:04,2020-03-07 01:21:04,"{""RWS01_MONIBAS_0201hrr0285ra"": 3000.0, ""RWS01_MONIBAS_0201hrr0284ra"": 2839.976189568402, ""RWS01_MONIBAS_0201hrr0257ra"": 431.05659007992244}","{""RWS01_MONIBAS_0201hrr0288ra"": 3231.814935862123, ""RWS01_MONIBAS_0201hrr0291ra"": 3533.7924432076416, ""RWS01_MONIBAS_0201hrr0295ra"": 3924.3419620541467, ""RWS01_MONIBAS_0201hrr0298ra"": 4231.080458587956, ""RWS01_MONIBAS_0201hrr0301ra"": 4614.416258398908, ""RWS01_MONIBAS_0200vwa0303ra"": 4743.437957210798, ""RWS01_MONIBAS_0201hrr0306ra"": 5026.462277661043, ""RWS01_MONIBAS_0201hrr0309ra"": 5396.736692551054, ""RWS01_MONIBAS_0201hrr0312ra"": 5673.210200071818, ""RWS01_MONIBAS_0201hrr0315ra"": 5961.9439339821965, ""RWS01_MONIBAS_0201hrr0319ra"": 6311.120786673314}"
4,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-10 02:42:35,2020-03-10 02:42:35,"{""RWS01_MONIBAS_0201hrr0285ra"": 3000.0, ""RWS01_MONIBAS_0201hrr0284ra"": 2839.976189568402, ""RWS01_MONIBAS_0201hrr0257ra"": 431.05659007992244}","{""RWS01_MONIBAS_0201hrr0288ra"": 3231.814935862123, ""RWS01_MONIBAS_0201hrr0291ra"": 3533.7924432076416, ""RWS01_MONIBAS_0201hrr0295ra"": 3924.3419620541467, ""RWS01_MONIBAS_0201hrr0298ra"": 4231.080458587956, ""RWS01_MONIBAS_0201hrr0301ra"": 4614.416258398908, ""RWS01_MONIBAS_0200vwa0303ra"": 4743.437957210798, ""RWS01_MONIBAS_0201hrr0306ra"": 5026.462277661043, ""RWS01_MONIBAS_0201hrr0309ra"": 5396.736692551054, ""RWS01_MONIBAS_0201hrr0312ra""

In [14]:
driver='postgresql'
username='dab_ds23241a_223'
dbname=username # it is the same as the username
password='Bh6KFgcWazMMCPvZ'
server='bronto.ewi.utwente.nl'
port='5432'

# Creating the connetcion pool for SQL
engine = create_engine(f'{driver}://{username}:{password}@{server}:{port}/{dbname}')
road_segments_data_df['before'] = road_segments_data_df['before'].apply(json.dumps)
road_segments_data_df['after'] = road_segments_data_df['after'].apply(json.dumps)
road_segments_data_df.to_sql('No_incidents_segments', engine,schema='project', index=False, if_exists='append')

80

In [16]:
# write road_segmenets_df to csv 
road_segments_data_df.to_csv('road_segments.csv', index=False)

# print min and max dates for the road segments
print(road_segments_data_df['starttijd'].min())
print(road_segments_data_df['starttijd'].max())

road_segments_data_df

2020-03-01 00:31:50
2020-03-28 01:09:42


,id,starttijd,before,after
0,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-10 03:18:55,2020-03-10 03:18:55,"""{\""RWS01_MONIBAS_0201hrr0285ra\"": 3000.0, \""RWS01_MONIBAS_0201hrr0284ra\"": 2839.976189568402, \""RWS01_MONIBAS_0201hrr0257ra\"": 431.05659007992244}""","""{\""RWS01_MONIBAS_0201hrr0288ra\"": 3231.814935862123, \""RWS01_MONIBAS_0201hrr0291ra\"": 3533.7924432076416, \""RWS01_MONIBAS_0201hrr0295ra\"": 3924.3419620541467, \""RWS01_MONIBAS_0201hrr0298ra\"": 4231.080458587956, \""RWS01_MONIBAS_0201hrr0301ra\"": 4614.416258398908, \""RWS01_MONIBAS_0200vwa0303ra\"": 4743.437957210798, \""RWS01_MONIBAS_0201hrr0306ra\"": 5026.462277661043, \""RWS01_MONIBAS_0201hrr0309ra\"": 5396.736692551054, \""RWS01_MONIBAS_0201hrr0312ra\"": 5673.210200071818, \""RWS01_MONIBAS_0201hrr0315ra\"": 5961.9439339821965, \""RWS01_MONIBAS_0201hrr0319ra\"": 6311.120786673314}"""
1,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-25 07:26:02,2020-03-25 07:26:02,"""{\""RWS01_MONIBAS_0201hrr0285ra\"": 3000.0, \""RWS01_MONIBAS_0201hrr0284ra\"": 2839.976189568402, \""RWS01_MONIBAS_0201hrr0257ra\"": 431.05659007992244}""","""{\""RWS01_MONIBAS_0201hrr0288ra\"": 3231.814935862123, \""RWS01_MONIBAS_0201hrr0291ra\"": 3533.7924432076416, \""RWS01_MONIBAS_0201hrr0295ra\"": 3924.3419620541467, \""RWS01_MONIBAS_0201hrr0298ra\"": 4231.080458587956, \""RWS01_MONIBAS_0201hrr0301ra\"": 4614.416258398908, \""RWS01_MONIBAS_0200vwa0303ra\"": 4743.437957210798, \""RWS01_MONIBAS_0201hrr0306ra\"": 5026.462277661043, \""RWS01_MONIBAS_0201hrr0309ra\"": 5396.736692551054, \""RWS01_MONIBAS_0201hrr0312ra\"": 5673.210200071818, \""RWS01_MONIBAS_0201hrr0315ra\"": 5961.9439339821965, \""RWS01_MONIBAS_0201hrr0319ra\"": 6311.120786673314}"""
2,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-22 06:43:22,2020-03-22 06:43:22,"""{\""RWS01_MONIBAS_0201hrr0285ra\"": 3000.0, \""RWS01_MONIBAS_0201hrr0284ra\"": 2839.976189568402, \""RWS01_MONIBAS_0201hrr0257ra\"": 431.05659007992244}""","""{\""RWS01_MONIBAS_0201hrr0288ra\"": 3231.814935862123, \""RWS01_MONIBAS_0201hrr0291ra\"": 3533.7924432076416, \""RWS01_MONIBAS_0201hrr0295ra\"": 3924.3419620541467, \""RWS01_MONIBAS_0201hrr0298ra\"": 4231.080458587956, \""RWS01_MONIBAS_0201hrr0301ra\"": 4614.416258398908, \""RWS01_MONIBAS_0200vwa0303ra\"": 4743.437957210798, \""RWS01_MONIBAS_0201hrr0306ra\"": 5026.462277661043, \""RWS01_MONIBAS_0201hrr0309ra\"": 5396.736692551054, \""RWS01_MONIBAS_0201hrr0312ra\"": 5673.210200071818, \""RWS01_MONIBAS_0201hrr0315ra\"": 5961.9439339821965, \""RWS01_MONIBAS_0201hrr0319ra\"": 6311.120786673314}"""
3,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-07 01:21:04,2020-03-07 01:21:04,"""{\""RWS01_MONIBAS_0201hrr0285ra\"": 3000.0, \""RWS01_MONIBAS_0201hrr0284ra\"": 2839.976189568402, \""RWS01_MONIBAS_0201hrr0257ra\"": 431.05659007992244}""","""{\""RWS01_MONIBAS_0201hrr0288ra\"": 3231.814935862123, \""RWS01_MONIBAS_0201hrr0291ra\"": 3533.7924432076416, \""RWS01_MONIBAS_0201hrr0295ra\"": 3924.3419620541467, \""RWS01_MONIBAS_0201hrr0298ra\"": 4231.080458587956, \""RWS01_MONIBAS_0201hrr0301ra\"": 4614.416258398908, \""RWS01_MONIBAS_0200vwa0303ra\"": 4743.437957210798, \""RWS01_MONIBAS_0201hrr0306ra\"": 5026.462277661043, \""RWS01_MONIBAS_0201hrr0309ra\"": 5396.736692551054, \""RWS01_MONIBAS_0201hrr0312ra\"": 5673.210200071818, \""RWS01_MONIBAS_0201hrr0315ra\"": 5961.9439339821965, \""RWS01_MONIBAS_0201hrr0319ra\"": 6311.120786673314}"""
4,no_incident_camera_baseRWS01_MONIBAS_0201hrr0319ra_2020-03-10 02:42:35,2020-03-10 02:42:35,"""{\""RWS01_MONIBAS_0201hrr0285ra\"": 3000.0, \""RWS01_MONIBAS_0201hrr0284ra\"": 2839.976189568402, \""RWS01_MONIBAS_0201hrr0257ra\"": 431.05659007992244}""","""{\""RWS01_MONIBAS_0201hrr0288ra\"": 3231.814935862123, \""RWS01_MONIBAS_0201hrr0291ra\"": 3533.7924432076416, \""RWS01_MONIBAS_0201hrr0295ra\"": 3924.3419620541467, \""RWS01_MONIBAS_0201hrr0298ra\"": 4231.080458587956, \""RWS01_MONIBAS_0201hrr0301ra\"": 4614.416258398908, \""RWS01_MONIBAS_0200